In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import random
import itertools
from itertools import product
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error
import warnings

In [2]:
%cd /data/p_dsi/teams2023/team7

/gpfs52/data/p_dsi/teams2023/team7


In [3]:
with open("dataframes_units_per_article_monthly.json") as json_file:
    data = json.load(json_file)

In [4]:
revert_dict = {}
for k in data.keys():
    temp = pd.DataFrame.from_dict(data[k])
    temp["DATE"] = pd.to_datetime(temp["DATE"])
    temp = temp.set_index("DATE")
    revert_dict.update({f"{k}": temp})
    
revert_dict

{'97980':              UNITS
 DATE              
 2015-04-30  2599.0
 2015-05-31  3587.0
 2015-06-30  3767.0
 2015-07-31  3323.0
 2015-08-31  3084.0
 2015-09-30  2618.0
 2015-10-31  3049.0
 2015-11-30  3454.0
 2015-12-31  3017.0
 2016-01-31  3117.0
 2016-02-29  2621.0
 2016-03-31  2877.0
 2016-04-30  2836.0
 2016-05-31  3113.0
 2016-06-30  3218.0
 2016-07-31  3098.0
 2016-08-31  2819.0
 2016-09-30  3559.0
 2016-10-31  3591.0
 2016-11-30  3771.0
 2016-12-31  4018.0
 2017-01-31  2953.0
 2017-02-28  2905.0
 2017-03-31  3494.0
 2017-04-30  3408.0
 2017-05-31  3422.0
 2017-06-30  3772.0
 2017-07-31  3490.0
 2017-08-31  3148.0
 2017-09-30  3454.0
 2017-10-31  3723.0
 2017-11-30  4294.0
 2017-12-31  3756.0
 2018-01-31  2897.0
 2018-02-28  2918.0
 2018-03-31  2946.0
 2018-04-30  2802.0
 2018-05-31  3237.0
 2018-06-30  3606.0
 2018-07-31  3208.0
 2018-08-31  3144.0
 2018-09-30  3325.0
 2018-10-31  3186.0,
 '97793':              UNITS
 DATE              
 2015-04-30  3525.0
 2015-05-31  4277.0
 

In [5]:
short_list = random.sample(list(revert_dict.keys()),3)
sample_dict = {k:revert_dict[k] for k in short_list}
print(sample_dict)

{'3193':              UNITS
DATE              
2015-04-30    76.0
2015-05-31    88.0
2015-06-30   110.0
2015-07-31   136.0
2015-08-31   128.0
2015-09-30   318.0
2015-10-31   217.0
2015-11-30   273.0
2015-12-31   230.0
2016-01-31   162.0
2016-02-29   207.0
2016-03-31   333.0
2016-04-30   453.0
2016-05-31   227.0
2016-06-30   189.0
2016-07-31   312.0
2016-08-31   399.0
2016-09-30   318.0
2016-10-31   463.0
2016-11-30   292.0
2016-12-31   298.0
2017-01-31   191.0
2017-02-28   166.0
2017-03-31   518.0
2017-04-30   387.0
2017-05-31   219.0
2017-06-30   216.0
2017-07-31   562.0
2017-08-31   590.0
2017-09-30   474.0
2017-10-31   371.0
2017-11-30   799.0
2017-12-31  1286.0
2018-01-31  1319.0
2018-02-28   809.0
2018-03-31   493.0
2018-04-30   608.0
2018-05-31   536.0
2018-06-30   449.0
2018-07-31   534.0
2018-08-31   626.0
2018-09-30   618.0
2018-10-31   339.0, '3040':              UNITS
DATE              
2017-08-31   211.0
2017-09-30   488.0
2017-10-31   688.0
2017-11-30   719.0
2017-12-31   

## Iterating through p,d,q

In [6]:
p = range(0,5)
q = range(0,5)
d = range(0,2)
pdq_comb = list(itertools.product(p,d,q))

In [7]:
best_models = {}
for article_id, data in sample_dict.items():
    best_model = None
    best_rmse = np.inf
    train = data[:len(data) - 2]
    test = data[len(data) - 2:len(data)]
    
    print(len(train),len(data)-1)

    for pdq in pdq_comb:
        try:
            # Fit an ARIMA model with the current pdq values
            model = ARIMA(train, order=pdq)
            results = model.fit()
            pred = results.predict(start=len(train),end =(len(data)-1))
            print(pdq)
            error = np.sqrt(mean_squared_error(test,pred))
            # Keep track of the model with the lowest rmse
            if error < best_rmse:
                best_rmse = error
                best_model = results
        #print(model.order)        
        except:
            continue
                
    best_models[article_id] = {'model': best_model, 'rmse': best_rmse}

41 42
(0, 0, 0)
(0, 0, 1)
(0, 0, 2)
(0, 0, 3)


/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/

(0, 0, 4)
(0, 1, 0)
(0, 1, 1)
(0, 1, 2)
(0, 1, 3)
(0, 1, 4)


/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/

(1, 0, 0)
(1, 0, 1)
(1, 0, 2)


...

/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization fail

(4, 0, 4)
(4, 1, 0)
(4, 1, 1)
(4, 1, 2)
(4, 1, 3)


/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/

(4, 1, 4)


/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [8]:
print(best_models)

{'3193': {'model': <statsmodels.tsa.arima.model.ARIMAResultsWrapper object at 0x2ad09b54a430>, 'rmse': 69.24225668522553}, '3040': {'model': <statsmodels.tsa.arima.model.ARIMAResultsWrapper object at 0x2ad09b582340>, 'rmse': 39.1571418226422}, '1432': {'model': <statsmodels.tsa.arima.model.ARIMAResultsWrapper object at 0x2ad09b21ef70>, 'rmse': 39.99113420550576}}


In [9]:
for article_id, model in best_models.items():
    print(f"Best model for article {article_id}: ARIMA {model['model'].model_orders}")

Best model for article 3193: ARIMA {'trend': 0, 'exog': 1, 'ar': 3, 'ma': 3, 'seasonal_ar': 0, 'seasonal_ma': 0, 'reduced_ar': 3, 'reduced_ma': 3, 'exog_variance': 0, 'measurement_variance': 0, 'variance': 1}
Best model for article 3040: ARIMA {'trend': 0, 'exog': 1, 'ar': 0, 'ma': 4, 'seasonal_ar': 0, 'seasonal_ma': 0, 'reduced_ar': 0, 'reduced_ma': 4, 'exog_variance': 0, 'measurement_variance': 0, 'variance': 1}
Best model for article 1432: ARIMA {'trend': 0, 'exog': 1, 'ar': 4, 'ma': 4, 'seasonal_ar': 0, 'seasonal_ma': 0, 'reduced_ar': 4, 'reduced_ma': 4, 'exog_variance': 0, 'measurement_variance': 0, 'variance': 1}


## Iterating through p,d,q & P,D,Q

In [ ]:
p_values = range(0, 3)
d_values = range(0, 2)
q_values = range(0, 3)
P_values = range(0, 3)
D_values = range(0, 2)
Q_values = range(0, 3)

results = []

#warnings.filterwarnings('ignore')

best_models = {}
for article_id, data in sample_dict.items():
    best_model = None
    best_rmse = np.inf
    train = data[:len(data) - 2]
    test = data[len(data) - 2:len(data)]
    
    print(len(train),len(data)-1)

    for p, d, q, P, D, Q in itertools.product(p_values, d_values, q_values, P_values, D_values, Q_values):

        # Skip if the non-seasonal order and seasonal order are both (0,0,0)
        if p == 0 and d == 0 and q == 0 and P == 0 and D == 0 and Q == 0:
            continue

        # Fit the ARIMA model with the current parameter combination
        try:
            model = ARIMA(data, order=(p, d, q), seasonal_order=(P, D, Q, 12))
            results = model.fit()
            pred = results.predict(start=len(train),end =(len(data)-1))
            error = np.sqrt(mean_squared_error(test,pred))
            # Keep track of the model with the lowest rmse
            if error < best_rmse:
                best_rmse = error
                best_model = results
        except:
            continue
    best_models[article_id] = best_model

/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting s

41 42


/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/

/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/

/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/

...

/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates

/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be se

/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates

/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be se

/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/accre/arch/

In [ ]:
best_models

In [ ]:
for article_id, model in best_models.items():
    #print(model.model_orders)
    print(f'Best model for article {article_id}: ARIMA{model.model_orders}')

In [ ]:
# Repeating the same procedure with different code 
# p_values = range(0, 4)
# d_values = range(0, 2)
# q_values = range(0, 4)
# P_values = range(0, 4)
# D_values = range(0, 2)
# Q_values = range(0, 4)
# s = 12

# # Find best SARIMA model for each article_id
# best_models = {}
# for article_id, data in sample_dict.items():
#     best_rmse = np.inf
#     best_model = None
#     for pdq, PDQS in product(product(p_values, d_values, q_values), product(P_values, D_values, Q_values, [s])):
#         try:
#             model = ARIMA(train, order=pdq, seasonal_order=PDQS)
#             results = model.fit()
#             pred = results.predict(start=len(train),end =(len(data)-1))
#             print(pdq)
#             error = np.sqrt(mean_squared_error(test,pred))
#             if error < best_rmse:
#                 best_rmse = error
#                 best_model = results
#         except:
#             continue
#     best_models[article_id] = {'model': best_model, 'rmse': best_rmse}

In [ ]:
# best_models

Conclusion:
- This notebook demonstrated the method to get p,d,q & P, D, Q in the ARIMA model through iteration
- It was relatively easy to get p, d, q values, but with the combinations of p,d,q & P, D, Q, the run time was too long
- Since 3 sampled article_ids were chosen in this notebook, it was observed that with 400+ article_id, we would encounter difficulties to perform the task to obtain p,d,q & P,D, Q values
- We would either go with the model with only p,d, and q values or chose different methods for choosing the parameters